In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from warnings import filterwarnings

In [2]:
filterwarnings('ignore')

In [3]:
data = pd.read_csv('heart.csv')

In [4]:
num_col = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']
cat_col = list(data.columns)
for col in num_col:
    cat_col.remove(col)

In [5]:
data_no_outliers = data
for col in num_col:
    min_thresh = data_no_outliers[col].mean() - 3 * data_no_outliers[col].std()
    max_thresh = data_no_outliers[col].mean() + 3 * data_no_outliers[col].std()
    data_no_outliers = data_no_outliers[(data_no_outliers[col] > min_thresh) & (data_no_outliers[col] < max_thresh)]

data = data_no_outliers

In [6]:
le = LabelEncoder()
for col in cat_col:
    data[col] = le.fit_transform(data[col])

In [7]:
target = data[['HeartDisease']]
data = data.drop(['HeartDisease'],axis='columns')

In [8]:
data = pd.get_dummies(data,columns=cat_col[:-1],dtype=int)

In [9]:
ss = StandardScaler()
data = ss.fit_transform(data)
target = target.to_numpy()

In [10]:
cross_val_score(SVC(),data,target,cv=10).mean()

0.8464169787765293

In [11]:
svm_bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    oob_score=True,
    max_samples=.8,
    random_state=7
)

cross_val_score(svm_bag,data,target,cv=10).mean()

0.8453058676654182

In [12]:
cross_val_score(DecisionTreeClassifier(),data,target,cv=10).mean()

0.7807865168539326

In [13]:
tree_bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    oob_score=True,
    max_samples=.8,
    random_state=7
)

cross_val_score(tree_bag,data,target,cv=10).mean()

0.8463795255930087